In [27]:

import numpy as np
import pynq
from pynq import Overlay
import time

# Load the overlay
overlay = Overlay("/home/ubuntu/EECS298_Final/part1/original_matrix/matmul_original.bit")

# Access the matrix multiplication IP instance in the overlay
matmul_inst = overlay.pl_matmul_a_0

# Define matrix sizes
M = 16
N = 16
K = 16

# Allocate memory for matrices A, B, and the result AB
matrix_A = pynq.allocate(shape=(M, K), dtype=np.int32)
matrix_B = pynq.allocate(shape=(K, N), dtype=np.int32)
matrix_AB = pynq.allocate(shape=(M, N), dtype=np.int32)

# Initialize matrices A and B with some values
for i in range(M):
    for j in range(K):
        matrix_A[i][j] = np.int32(i + j)

for i in range(K):
    for j in range(N):
        matrix_B[i][j] = np.int32(i - j)

# Initialize matrix AB with -1.0
for i in range(M):
    for j in range(N):
        matrix_AB[i][j] = np.int32(-1)

# Synchronize data to the FPGA
matrix_A.sync_to_device()
matrix_B.sync_to_device()
matrix_AB.sync_to_device()

# Set the physical addresses of matrices A, B, and AB
matmul_inst.mmio.write_reg(0x10, matrix_A.physical_address)
matmul_inst.mmio.write_reg(0x14, 0)
matmul_inst.mmio.write_reg(0x1C, matrix_B.physical_address)
matmul_inst.mmio.write_reg(0x20, 0)
matmul_inst.mmio.write_reg(0x28, matrix_AB.physical_address)
matmul_inst.mmio.write_reg(0x2C, 0)

# Measure execution time for plain accelerator
start_time_plain = time.time()

# Trigger the matrix multiplication IP
matmul_inst.write(0x00, 1)

# Wait for the accelerator to finish
while matmul_inst.read(0x00) == 14:
    pass

end_time_plain = time.time()
execution_time_plain = end_time_plain - start_time_plain

# Synchronize data back from the FPGA
matrix_A.sync_from_device()
matrix_B.sync_from_device()
matrix_AB.sync_from_device()

# Print results and execution time for plain accelerator
print("Matrix A:")
print(matrix_A)
print("Matrix B:")
print(matrix_B)
print("Result AB (Plain Accelerator):")
print(matrix_AB)
print(f"Execution Time (Plain Accelerator): {execution_time_plain:.6f} seconds")


# Release allocated memory
matrix_A.freebuffer()
matrix_B.freebuffer()
matrix_AB.freebuffer()

Matrix A:
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
 [ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
 [ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
 [ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
 [ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
 [ 7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
 [ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
 [ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
 [10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25]
 [11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26]
 [12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27]
 [13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]
 [14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
 [15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30]]
Matrix B:
[[  0  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14 -15]
 [  1   0  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14]
 [  2   1   0  -1  -2  -3  -4

In [26]:
ip_dict = overlay.ip_dict

In [18]:
for ip_name in ip_dict.keys():
    print("IP Block Name:", ip_name)

IP Block Name: pl_matmul_opt_0
IP Block Name: zynq_ultra_ps_e_0
